In [1]:
from  langchain_openai import ChatOpenAI
import os 
from dotenv import load_dotenv

# This function will load all the variables from the .env file and will 
# make them available in the os.environ dictionary (env variables)
load_dotenv() 

if os.environ.get("OPENAI_API_KEY"):
    print("Bro API KEY Variable exists")
else:
    raise ValueError("OPENAI_API_KEY not found")

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from  langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm_openai = ChatOpenAI(model="gpt-5-mini",temperature=0)

Bro API KEY Variable exists


# **CHAIN WITH Parallel Chains**

In [2]:
# TASK -1 [Prompt]

prompt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a movie summarizer"),
    ("human", "Please summarize the movie in brief : {input}")])

In [3]:
# TASK - 2 [LLM]

llm_openai = ChatOpenAI(model="gpt-5-mini",temperature=0)

In [4]:
# TASK - 3 [Str Parser]

str_parser = StrOutputParser()

In [6]:
# TASK - 4 [Custom Runnable]
from langchain_core.runnables import RunnableLambda

def dictionary_maker(text:str)-> dict:

    return {"text" : text}

dictionary_maker_runnable = RunnableLambda(dictionary_maker)

### **Parallel Chain 1**

In [7]:
# TASK - 1 [Prompt]

linkedin_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a LinkedIn post generator"),
    ("human", "Create a post for the following text for LinkedIn: {text}")])

# TASK - 2 [LLM]

llm_openai = ChatOpenAI(model="gpt-5-mini",temperature=0)

# TASK - 3 [Str Parser]

str_parser = StrOutputParser()

chain_linkedin = linkedin_prompt | llm_openai | str_parser

### **Parallel Chain 2**

In [9]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

In [8]:
def insta_chain(text:dict):

    text = text["text"]

    # TASK - 1 [Prompt]
    insta_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a LinkedIn post generator"),
    ("human", "Create a post for the following text for Instagram: {text}")])
    
    # TASK - 2 [LLM]
    llm_openai = ChatOpenAI(model="gpt-5-mini",temperature=0)

    # TASK - 3 [Str Parser]
    str_parser = StrOutputParser()

    chain_insta = insta_prompt | llm_openai | str_parser

    result = chain_insta.invoke(text)

    return result

insta_chain_runnable = RunnableLambda(insta_chain)


### **Final Orchestration**

In [10]:
final_chain = ( 
                    prompt_template | 
                    llm_openai | 
                    str_parser | 
                    dictionary_maker_runnable |
                    RunnableParallel(branches = {"linkedin": chain_linkedin, "instagram": insta_chain_runnable})
)
    

In [11]:
final_chain.invoke("KGF")

{'branches': {'linkedin': 'Watched KGF recently — and beyond the high‑octane action, it’s a compact lesson in ambition, leadership and the costs of power.\n\nQuick summary: Rocky rises from Bombay’s slums with one clear purpose — to keep a promise to his dying mother. His search for wealth and influence takes him to the brutal Kolar Gold Fields, where he infiltrates a lawless system, becomes a charismatic (and feared) leader, confronts corrupt forces, and fights to change the balance of power. The story continues in KGF: Chapter 2.\n\nThree leadership takeaways:\n- Purpose fuels perseverance: a vivid north star can drive extraordinary resilience and risk‑taking.\n- Leadership is a double‑edged sword: charisma and decisiveness can inspire loyalty, but when paired with ruthless tactics they create ethical tradeoffs and long‑term instability.\n- Systems matter: changing outcomes often requires confronting entrenched power structures — strategy and alliances matter as much as courage.\n\nK

## **Chain as a Runnable**

In [13]:
# TASK - 1 [Beautify Function]

def beautify(final_response:dict)-> dict:

    linkedin_response = final_response['branches']['linkedin']
    instagram_response = final_response['branches']['instagram']

    return {"linkedin": linkedin_response, "instagram": instagram_response}

beautify_runnable = RunnableLambda(beautify)


# TASK - 2 [Final Chain]

# final_chain 


# Beautified Chain
beautified_chain = final_chain | beautify_runnable

beautified_chain.invoke("Pushpa")


{'linkedin': 'Just watched Pushpa: The Rise (2021), directed by Sukumar — a gritty Telugu action‑drama that’s as much about survival and ambition as it is about crime.\n\nQuick snapshot:\n- Follows Pushpa Raj (Allu Arjun), a lowly labourer in the Seshachalam forests who claws his way up the red sandalwood trade through grit, cunning and violence.\n- As he gains power he clashes with rivals, corrupt officials and the law, while negotiating a fraught personal relationship with Srivalli.\n- Noted for a standout lead performance, driving music and high‑energy action.\n\nTakeaways for professionals and storytellers:\n- Ambition + resourcefulness can change trajectories — but so do the choices and trade‑offs along the way.\n- Systems matter: understanding informal rules, incentives and power structures is crucial when operating in complex environments.\n- Leadership is messy: gaining influence brings conflicts, ethical dilemmas and the need to manage alliances.\n- Craft counts: execution — p